In [2]:
import numpy as np
data = np.load('phased/chr.22.phased.npz')
all_states, all_loss = data['states'], data['losses']

In [7]:
print(np.where(all_states[0, :] != 0))

(array([  57941,   57944,   57946, ..., 1004639, 1004640, 1004642]),)
